This is for implementing CNN in order to determine if string contains hologram or not

In [1]:
import pandas as pd
import zipfile
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

Upload our data from google drive

In [2]:
# Mount google drive
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# Specify the path to the ZIP file
zip_file_path = "/content/gdrive/MyDrive/MASTER/TER/generatedImages.zip"

# Extract the contents of the ZIP file to a specified directory
with zipfile.ZipFile(zip_file_path, 'r') as zipped_data:
    zipped_data.extractall("/temp")

In [4]:
# we'll see if needed
# Define transforms to preprocess the data (you can customize these as needed)
transform = transforms.Compose([
    transforms.Pad((180, 0, 211, 0), fill=0),  # Add Padding:  Our images are 64x33 (left,right,top, bottom)
     transforms.Resize((244, 244)),
    #transforms.CenterCrop(224),         # Crop the center 224x224 portion of the image
    transforms.ToTensor(),              # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

In [11]:
# Load the dataset using ImageFolder
dataset = datasets.ImageFolder(root="/temp", transform=transform)

# The size for each data split
train_size = int(0.001*len(dataset))
validation_size = int(0.899*len(dataset))
test_size = int(0.1*len(dataset))

# Use random_split to split the dataset into train, validation, and test sets
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, validation_size, test_size])


# Define DataLoader for each split using the respective sampler
training = DataLoader(train_dataset, batch_size=128, shuffle=True )
validation = DataLoader(val_dataset, batch_size=128, shuffle=True)
testing = DataLoader(test_dataset, batch_size=128, shuffle=True)

In [12]:
# Define your CNN architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64*61*61, 128)
        self.fc2 = nn.Linear(128, 2)  # Output 2 classes: 0 or 1

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [13]:
# Instantiate the CNN model
model = CNN()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in training:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(training)}")

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testing:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}%")

Epoch 1/10, Loss: 0.6886025071144104
Epoch 2/10, Loss: 9.388144493103027
Epoch 3/10, Loss: 22.148944854736328
Epoch 4/10, Loss: 9.159502029418945
Epoch 5/10, Loss: 0.5240204930305481
Epoch 6/10, Loss: 2.5209641456604004
Epoch 7/10, Loss: 1.9644497632980347
Epoch 8/10, Loss: 1.0095784664154053
Epoch 9/10, Loss: 0.5593355298042297
Epoch 10/10, Loss: 0.6702460050582886
Accuracy: 48.833333333333336%
